In [1]:
import re
import codecs
import string
import operator
import functools
import glob
import os
import matplotlib.colors as mc
import nltk
import numpy as np
import textstat
import unicodedata
import pandas as pd
import math
import textstat
import numpy as np
import fnmatch

#nltk.install("all")

from lexicalrichness import LexicalRichness
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import sent_tokenize

from seg import Segmentation

 # Feature Selection
from sklearn.feature_selection import VarianceThreshold

# os.remove(path)
# if os.path.exists(path):
#    os.remove(path)

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 10 19:16:37 2020

@author: pablonicolasr
"""
from dataset import generate
from cluster import cluster_ip
from lxml import etree


actual_directory = os.getcwd()

ruta = os.path.join(actual_directory, "mini_corpus")
ruta_del_directorio = str(ruta) + "\\*.txt"
# Me produce una lista con los nombres de los archivos txt del corpus
textos_originales = glob.glob(ruta_del_directorio)


if not os.path.exists(str(actual_directory)+"\\cleaned_corpus"):
    os.mkdir(str(actual_directory)+'\\cleaned_corpus')
    
cleaned_corpus = str(actual_directory)+"\\cleaned_corpus"

# De esta manera le voy robo el nombre a los xml
path_xml = str(actual_directory) + "\\mini_corpus" + "\\*.xml"
xml_docs = glob.glob(path_xml)


if not os.path.exists(str(actual_directory)+"\\sospechoso"):
    os.mkdir(str(actual_directory)+'\\sospechoso')
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos")
    patron_inicio = 'sosp*'
    patron_final = '*.txt'
    archivos = os.listdir(str(actual_directory)+"\\sospechoso")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_inicio) and fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory)+"\\sospechoso", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")

    
if not os.path.exists(str(actual_directory) + "\\xml\\output_xml"):
    os.mkdir(str(actual_directory) + "\\xml\\output_xml")
else:
    print("Se elimino carpeta vieja de fragmentos sospechosos en formato xml")
    patron_final = '*.xml'
    archivos = os.listdir(str(actual_directory) + "\\xml\\output_xml")
    for archivo in archivos:
        if fnmatch.fnmatch(archivo, patron_final):
            ruta_completa = os.path.join(str(actual_directory) + "\\xml\\output_xml", archivo)
            os.remove(ruta_completa)
            print(f"Eliminado: {ruta_completa}")


sospechosos_path = str(actual_directory) + "\\sospechoso\\"
output_xml = str(actual_directory) + "\\xml\\output_xml\\"


f = {}
# Comienzo a leer el conjunto de textos
for i in range(len(textos_originales)):
    print("File: " + str(i+1))
    f[i] = open(textos_originales[i], encoding= "utf-8-sig")
    text = f[i].read()
    f[i].close()            
    
    # Nombre de Archivo
    filename = os.path.basename(textos_originales[i])
    # Separo nombre de archivo y extension
    filename2, extension = os.path.splitext(filename)
    
    Seg = Segmentation(text)
    # La variable data_text posee el texto completo, separado en párrafos
    data_text = Seg.paraSegmentation()
    data_list = []
    length = []
    
    # Se tiene guardar el texto sin limpiar
    for segmento in data_text:
        # A cada segmento se le borra el salto de línea
        segmento = re.sub("\n", " ", segmento)
        # Se guarda la longitud del segmento
        longitud = len(segmento)
        length.append(longitud)
        # Se guarda el segmento
        data_list.append(segmento)
    
    # De aqui empezar a procesar la limpieza de texto 
    # Some code...
    # .....
    # .....
    
    rootPath = ruta + "\\"+ filename
    outputPath = str(actual_directory) + "\\csv\\"+ filename2 + ".csv"
    
    if os.path.exists(str(actual_directory) + "\\csv\\"+ filename2 + ".csv"):
        os.remove(str(actual_directory) + "\\csv\\"+ filename2 + ".csv")
    
    generate(rootPath, outputPath, filename, "english")
    
    df = pd.read_csv(outputPath)
    
    # Jugar con estos números // Tarea
    
    # samples = len(length) * 0.075
    
    # decimal, entero = math.modf(samples)
    
    # if decimal >= 0.5:
        # min_samples = entero + 1
    # else:
        # min_samples = entero
    
    # Este número fue sacado de un paper
    min_samples = 4
    
    # Normalizar de forma correcta y eficiente el indice de facilidad de lectura
    df["fleshReadingEase"] = df["fleshReadingEase"] / df["fleshReadingEase"].max()
    
    # Seleccionar los valores numéricos del df
    # Este paso se realiza para ir viendo que genera cada párrafo del txt y como fue convertido a un vector
    dataframe = df.iloc[:,[2,3,4]]
    
    # Aprendizaje No Supervisado
    cluster = cluster_ip(dataframe, min_samples, "cosine")
    labels = cluster.labels_    
    df["labels"] = labels
    
    realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
    clusterNum=len(set(labels))
    n_noise = list(labels).count(-1) 
    
    print("Estimated number of clusters: %d" % realClusterNum)
    print("Estimated number of noise points: %d" % n_noise)
    counts = df["labels"].value_counts().to_dict()
    print(counts)
    
    # Me sirve para guardar los casos de plagio y compararlos
    xml_name = os.path.basename(xml_docs[i])
    xml_name2, extension = os.path.splitext(xml_name)
    
    # Se crea el archivo txt donde se guardaran los fragmentos sospechosos
    sherlock = codecs.open(sospechosos_path + "sosp" + filename,"w","utf-8")
    
    # Los fragmentos que fueron clasificados como plagio
    # Se guardan en el archivo .txt
    # Tener en cuenta que el dataframe y la lista data_list, manejan los mismos índices
    # Por eso, si en el dataframe se encuentra un "-1" es porque este es un outlier
    for x in range(len(df["labels"])):
        l = df["labels"][x]
        if l == -1:
            print(data_list[x])
            print(x)
            sherlock.write(data_list[x]+os.linesep)       
               
    sherlock.close()    
    
    document_el = etree.Element("document", reference = xml_name2 + ".txt")
    
    offset = 0
    
    for y in range(len(df["labels"])):       
        
        l = df["labels"][y]
        
        if l == -1: 
            
            feature_el = etree.SubElement(document_el, "feature", name="detected-plagiarism", this_offset=str(offset), this_length=str(length[y]))
    
        offset += length[y]
        
        print(f"{xml_name2}")
        
        print(f"Offset: {offset}")
                          
            
    et = etree.ElementTree(document_el)
    with open(output_xml+xml_name, "wb") as fe:
        et.write(fe, encoding="utf-8", xml_declaration=None, pretty_print=True)

Se elimino carpeta vieja de fragmentos sospechosos
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00001.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00002.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00003.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00004.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00005.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00006.txt
Eliminado: C:\Users\pablonicolasr\Desktop\pablonicolas\mentodatos\plagiarism\practico3\sospechoso\sospsuspicious-document00007.txt
Eliminado: C:\Users\pablonicolas

Row: [51, 'suspicious-document00002.txt', 63.9, 0.1258741258741259, 0.6850393700787402]
Row: [52, 'suspicious-document00002.txt', 60.58, 0.16149068322981366, 0.7142857142857143]
Row: [53, 'suspicious-document00002.txt', 60.65, 0.16417910447761194, 0.8070175438596491]
Row: [54, 'suspicious-document00002.txt', 33.41, 0.11764705882352941, 0.7466666666666667]
Row: [55, 'suspicious-document00002.txt', 77.91, 0.6666666666666666, 1.0]
Row: [56, 'suspicious-document00002.txt', 93.81, 0.4, 1.0]
Row: [57, 'suspicious-document00002.txt', 77.91, 0.3333333333333333, 1.0]
Row: [58, 'suspicious-document00002.txt', 35.61, 0.3333333333333333, 1.0]
Row: [59, 'suspicious-document00002.txt', 5.84, 0.17045454545454544, 0.7466666666666667]
Row: [60, 'suspicious-document00002.txt', 17.88, 0.1322314049586777, 0.589622641509434]
Row: [61, 'suspicious-document00002.txt', 49.49, 0.1188118811881188, 0.7582417582417582]
Row: [62, 'suspicious-document00002.txt', 44.11, 0.1092436974789916, 0.6792452830188679]
Row: [

Row: [185, 'suspicious-document00002.txt', 40.01, 0.225, 0.7096774193548387]
Row: [186, 'suspicious-document00002.txt', 17.51, 0.14184397163120568, 0.6991869918699187]
Row: [187, 'suspicious-document00002.txt', 77.91, 0.3333333333333333, 1.0]
Row: [188, 'suspicious-document00002.txt', 75.88, 0.2, 1.0]
Row: [189, 'suspicious-document00002.txt', 46.78, 0.13380281690140844, 0.47368421052631576]
Row: [190, 'suspicious-document00002.txt', 20.73, 0.1286549707602339, 0.5165562913907285]
Row: [191, 'suspicious-document00002.txt', 13.62, 0.06557377049180328, 0.6403508771929824]
Row: [192, 'suspicious-document00002.txt', 69.11, 0.1827956989247312, 0.7142857142857143]
Row: [193, 'suspicious-document00002.txt', 28.85, 0.10638297872340426, 0.8333333333333334]
Row: [194, 'suspicious-document00002.txt', 36.7, 0.11428571428571428, 0.5601173020527859]
Row: [195, 'suspicious-document00002.txt', 60.18, 0.15656565656565657, 0.6826347305389222]
Row: [196, 'suspicious-document00002.txt', 69.31, 0.1374045801

Row: [343, 'suspicious-document00002.txt', 49.83, 0.13513513513513514, 0.5826086956521739]
Row: [344, 'suspicious-document00002.txt', 65.05, 0.11538461538461539, 0.8043478260869565]
Row: [345, 'suspicious-document00002.txt', 74.9, 0.2, 0.775]
Row: [346, 'suspicious-document00002.txt', 61.6, 0.11944444444444445, 0.5849056603773585]
Row: [347, 'suspicious-document00002.txt', 65.05, 0.12380952380952381, 0.7362637362637363]
Row: [348, 'suspicious-document00002.txt', 35.61, 0.3333333333333333, 1.0]
Row: [349, 'suspicious-document00002.txt', 54.86, 0.10843373493975904, 0.76]
Row: [350, 'suspicious-document00002.txt', 26.48, 0.18181818181818182, 0.726027397260274]
Row: [351, 'suspicious-document00002.txt', 51.31, 0.14507772020725387, 0.6428571428571429]
Row: [352, 'suspicious-document00002.txt', 53.55, 0.13333333333333333, 0.84]
Row: [353, 'suspicious-document00002.txt', 69.45, 0.16923076923076924, 0.7407407407407407]
Row: [354, 'suspicious-document00002.txt', -6.7, 0.3333333333333333, 1.0]
R

Row: [30, 'suspicious-document00003.txt', 62.04, 0.09271523178807947, 0.6474820143884892]
Row: [31, 'suspicious-document00003.txt', 73.85, 0.0, 1.0]
Row: [32, 'suspicious-document00003.txt', 63.87, 0.09722222222222222, 0.78125]
Row: [33, 'suspicious-document00003.txt', 36.12, 0.10526315789473684, 0.5980392156862745]
Row: [34, 'suspicious-document00003.txt', 63.53, 0.12171052631578948, 0.5350553505535055]
Row: [35, 'suspicious-document00003.txt', 55.61, 0.11926605504587157, 0.5897435897435898]
Row: [36, 'suspicious-document00003.txt', 62.41, 0.11805555555555555, 0.6666666666666666]
Row: [37, 'suspicious-document00003.txt', 68.5, 0.12179487179487179, 0.6194029850746269]
Row: [38, 'suspicious-document00003.txt', 63.36, 0.1111111111111111, 1.0]
Row: [39, 'suspicious-document00003.txt', 34.26, 0.11333333333333333, 0.6258992805755396]
Row: [40, 'suspicious-document00003.txt', 66.07, 0.1, 0.8409090909090909]
Row: [41, 'suspicious-document00003.txt', 63.36, 0.1111111111111111, 1.0]
Row: [42, '

...

Row: [173, 'suspicious-document00010.txt', 63.29, 0.11914893617021277, 0.4782608695652174]
Row: [174, 'suspicious-document00010.txt', 67.76, 0.09016393442622951, 0.6036036036036037]
Row: [175, 'suspicious-document00010.txt', 64.88, 0.11320754716981132, 0.6031746031746031]
Row: [176, 'suspicious-document00010.txt', 79.13, 0.13924050632911392, 0.587378640776699]
Row: [177, 'suspicious-document00010.txt', 57.84, 0.12454212454212454, 0.6348547717842323]
Row: [178, 'suspicious-document00010.txt', 52.57, 0.09615384615384616, 0.5815602836879432]
Row: [179, 'suspicious-document00010.txt', 65.56, 0.11764705882352941, 0.8]
Row: [180, 'suspicious-document00010.txt', 54.39, 0.15384615384615385, 0.8059701492537313]
Row: [181, 'suspicious-document00010.txt', 65.76, 0.06944444444444445, 0.7611940298507462]
Row: [182, 'suspicious-document00010.txt', -6.7, 0.3333333333333333, 1.0]
Row: [183, 'suspicious-document00010.txt', -8.73, 0.2, 1.0]
Row: [184, 'suspicious-document00010.txt', 59.71, 0.11724137931

Row: [325, 'suspicious-document00010.txt', 70.02, 0.12195121951219512, 0.6712328767123288]
Row: [326, 'suspicious-document00010.txt', 48.77, 0.08955223880597014, 0.6358695652173914]
Row: [327, 'suspicious-document00010.txt', 46.68, 0.09047619047619047, 0.4774305555555556]
Row: [328, 'suspicious-document00010.txt', 38.32, 0.16666666666666666, 0.8536585365853658]
Row: [329, 'suspicious-document00010.txt', 62.18, 0.12371134020618557, 0.7411764705882353]
Row: [330, 'suspicious-document00010.txt', 76.22, 0.14634146341463414, 0.8055555555555556]
Row: [331, 'suspicious-document00010.txt', 49.83, 0.09973753280839895, 0.5685131195335277]
Row: [332, 'suspicious-document00010.txt', 58.96, 0.08421052631578947, 0.7931034482758621]
Row: [333, 'suspicious-document00010.txt', 80.24, 0.21739130434782608, 0.7702702702702703]
Row: [334, 'suspicious-document00010.txt', 49.83, 0.10377358490566038, 0.6684210526315789]
Row: [335, 'suspicious-document00010.txt', 58.79, 0.12352941176470589, 0.6733333333333333]

Row: [475, 'suspicious-document00010.txt', 65.8, 0.12, 0.5795918367346938]
Row: [476, 'suspicious-document00010.txt', 57.95, 0.12121212121212122, 0.7333333333333333]
Row: [477, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [478, 'suspicious-document00010.txt', 45.42, 0.1, 0.7777777777777778]
Row: [479, 'suspicious-document00010.txt', 61.43, 0.08955223880597014, 0.5755102040816327]
Row: [480, 'suspicious-document00010.txt', 61.7, 0.15053763440860216, 0.620253164556962]
Row: [481, 'suspicious-document00010.txt', 42.38, 0.11377245508982035, 0.6283783783783784]
Row: [482, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [483, 'suspicious-document00010.txt', 92.8, 0.2, 1.0]
Row: [484, 'suspicious-document00010.txt', 32.74, 0.09803921568627451, 0.7419354838709677]
Row: [485, 'suspicious-document00010.txt', 56.08, 0.22413793103448276, 0.8085106382978723]
Row: [486, 'suspicious-document00010.txt', 49.99, 0.10606060606060606, 0.7796610169491526]
Row: [

Row: [648, 'suspicious-document00010.txt', 68.84, 0.12307692307692308, 0.3553459119496855]
Row: [649, 'suspicious-document00010.txt', 63.56, 0.11320754716981132, 0.431924882629108]
Row: [650, 'suspicious-document00010.txt', 77.91, 0.3333333333333333, 1.0]
Row: [651, 'suspicious-document00010.txt', 100.24, 0.16666666666666666, 0.8]
Row: [652, 'suspicious-document00010.txt', 64.88, 0.10426540284360189, 0.5767195767195767]
Row: [653, 'suspicious-document00010.txt', 61.63, 0.10344827586206896, 0.6952380952380952]
Row: [654, 'suspicious-document00010.txt', 71.78, 0.11700468018720749, 0.4842105263157895]
Row: [655, 'suspicious-document00010.txt', 58.15, 0.07453416149068323, 0.6510067114093959]
Row: [656, 'suspicious-document00010.txt', 49.28, 0.14772727272727273, 0.695364238410596]
Row: [657, 'suspicious-document00010.txt', 75.47, 0.12251655629139073, 0.48120300751879697]
Row: [658, 'suspicious-document00010.txt', 27.33, 0.11612903225806452, 0.656934306569343]
Row: [659, 'suspicious-document

Row: [808, 'suspicious-document00010.txt', 120.21, 0.3333333333333333, 1.0]
Row: [809, 'suspicious-document00010.txt', 52.8, 0.10344827586206896, 0.5846153846153846]
Row: [810, 'suspicious-document00010.txt', 46.61, 0.18333333333333332, 0.717391304347826]
Row: [811, 'suspicious-document00010.txt', 22.08, 0.23809523809523808, 0.967741935483871]
Row: [812, 'suspicious-document00010.txt', 70.43, 0.2222222222222222, 0.74]
Row: [813, 'suspicious-document00010.txt', 67.76, 0.1111111111111111, 0.8695652173913043]
Row: [814, 'suspicious-document00010.txt', 62.01, 0.07142857142857142, 0.84]
Row: [815, 'suspicious-document00010.txt', 25.97, 0.1347517730496454, 0.6229508196721312]
Row: [816, 'suspicious-document00010.txt', 27.5, 0.07692307692307693, 0.7758620689655172]
Row: [817, 'suspicious-document00010.txt', 81.63, 0.11764705882352941, 1.0]
Row: [818, 'suspicious-document00010.txt', 80.92, 0.16265060240963855, 0.5285714285714286]
Row: [819, 'suspicious-document00010.txt', 60.69, 0.091666666666